In [2]:
import pandas as pd
import numpy as np

### Địa chỉ file gốc và file đích

#### Dự án

In [3]:
# path_data_original = '../../Data/originalData/data_project_new.csv'
# path_data_goal = '../../Data/cleanedData/cleaned_data_project.csv'

#### Toàn bộ

In [4]:

path_data_original = '../../Data/originalData/data_original_new.csv'
path_data_goal = '../../Data/cleanedData/cleaned_data_new.csv'

In [5]:
data = pd.read_csv(path_data_original)

### Drop những căn bị thiếu thông tin



In [6]:
data = data.drop_duplicates()
data = data[data['Tỉnh/Thành phố'].str.contains('Hà Nội', case=False, na=False)].reset_index(drop=True)
data = data[~data['Chủ đầu tư'].str.contains('Đang cập nhật', case=False, na=False)].reset_index(drop=True)
data = data[~data['Tên dự án'].str.contains('Đang cập nhật', case=False, na=False)].reset_index(drop=True)
data = data[~data['Mức giá'].str.contains(r'Thỏa thuận|nghìn', case=False, na=False)].reset_index(drop=True)
data = data[~data['Số phòng ngủ'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Số toilet'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Pháp lý'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)

Sửa form và chuẩn hóa (Mức độ của Pháp lý và Nội thất)

In [7]:
import unicodedata

# Hàm chuẩn hóa Unicode
def normalize_unicode(text):
    if pd.isna(text):  # Kiểm tra giá trị null
        return text
    return unicodedata.normalize('NFKC', text)

# Áp dụng chuẩn hóa cho các cột
data['Quận/Huyện'] = data['Quận/Huyện'].apply(normalize_unicode).str.replace(r'Quận|Huyện', '', regex=True).str.strip().str.title()
data['Xã/Phường'] = data['Xã/Phường'].apply(normalize_unicode).str.replace(r'Xã|Phường|Thị trấn', '', regex=True).str.strip().str.title()


In [8]:
len(data)

11240

In [9]:
data['Quận/Huyện'].value_counts().sort_values()

Quận/Huyện
Phú Xuyên          5
Mê Linh            5
Mỹ Đức            11
Ứng Hòa           15
Phúc Thọ          16
Sơn Tây           18
Ba Vì             18
Thường Tín        23
Thạch Thất        26
Đan Phượng        44
Thanh Oai         83
Sóc Sơn           84
Nam Từ Liêm      116
Đông Anh         127
Hoài Đức         152
Chương Mỹ        157
Quốc Oai         158
Hoàn Kiếm        290
Thanh Trì        362
Gia Lâm          481
Đống Đa          560
Bắc Từ Liêm      694
Ba Đình          707
Long Biên        779
Tây Hồ           835
Hai Bà Trưng     880
Hoàng Mai        883
Cầu Giấy        1126
Thanh Xuân      1137
Hà Đông         1448
Name: count, dtype: int64

In [10]:
type = data['Phân loại']
for form in ['Căn hộ chung cư', 'Chung cư mini, căn hộ dịch vụ', 'Nhà riêng', 'Nhà Biệt thự, liền kề', 'Nhà mặt phố',
            'Shophouse, nhà phố thương mại', 'Đất nền dự án', 'Bán đất', 'Condotel', 'Trang trại, khu nghỉ dưỡng', 'Kho, nhà xưởng', 'Bất động sản khác']:
    wrong_form = type.str.contains(form, case=False, na=False)
    data.loc[wrong_form, 'Phân loại'] = form


In [11]:
area = data['Diện tích']
area = area.str.replace(' m²', '').str.replace('.', '').str.replace(',', '.').astype(np.float32)

In [12]:
price = data['Mức giá']

wrong_form = price.str.contains('/m²', case=False, na=False)
cleaned_wrong_form = data.loc[wrong_form, 'Mức giá'].str.replace('/m²', '').str.replace(',', '.')
data.loc[wrong_form, 'Mức giá'] = cleaned_wrong_form

price_area = price.str.contains('tỷ', case=False, na=False)
cleaned_price_area = data.loc[price_area, 'Mức giá'].str.replace(' tỷ', '').str.replace(',', '.').astype(np.float32)*1000
price_per_square = (cleaned_price_area / area[price_area])
data.loc[price_area, 'Mức giá'] = price_per_square

wrong_form = price.str.contains(' triệu', case=False, na=False)
cleaned_wrong_form = data.loc[wrong_form, 'Mức giá'].str.replace(' triệu', '').str.replace(',', '.').astype(np.float32)
data.loc[wrong_form, 'Mức giá'] = cleaned_wrong_form

In [13]:
bed_room = data['Số phòng ngủ']
bed_room = bed_room.str.replace(' phòng', '').astype(np.float32)
toilet_room = data['Số toilet']
toilet_room = toilet_room.str.replace(' phòng', '').astype(np.float32)

In [14]:
legal = data['Pháp lý']

wrong_form = legal.str.contains('.', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = data.loc[wrong_form, 'Pháp lý'].str.replace('.', '')

wrong_form = legal.str.contains(r'hợp đồng|HĐMB|HDMB|mua bán', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = 3 # "Hợp đồng mua bán"

wrong_form = legal.str.contains('ủy quyền', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = 2 # "ủy quyền"

wrong_form = legal.str.contains(r'đang chờ sổ|vi bằng', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = 1 #"Chưa có"

wrong_form = legal.str.contains(r'sổ|so|Sổ đỏ/ Sổ hồng|đầy đủ|Đủ|hợp lệ|Số đỏ|sạch|sang tên|phân lô|xã|quân đội|Pháp lý', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = 4 #'Đầy đủ'

In [15]:
data['Pháp lý'].drop_duplicates()

0       4
7       3
247     1
9434    2
Name: Pháp lý, dtype: object

In [16]:
furniture = data['Nội thất']
wrong_form = furniture.str.contains('.', case=False, na=False)
data.loc[wrong_form, 'Nội thất'] = data.loc[wrong_form, 'Nội thất'].str.replace('.', '')

wrong_form = furniture.str.contains(r'Cao cấp|5|đẳng cấp|SIÊU VÍP|Co cấp|tiền tỉ|VIP|cáo cấp|đẹp|ngoại|semi|hiện đại|sang|nhập khẩu|xịn|châu âu|tâm huyết|hien dai', case=False, na=False)
data.loc[wrong_form, 'Nội thất'] = 4 #'Cao cấp'

wrong_form = furniture.str.contains(r'cơ bản|tặng|ổn|NTCB|nguyên bản|nhà mới|điều|mới|thang máy|kèm nội thất|41518075|hầm chìm', case=False, na=False)
data.loc[wrong_form, 'Nội thất'] = 2 #'Cơ bản'

wrong_form = furniture.str.contains(r'Đầy đủ|Nội thất:|như ảnh|chủ đầu tư|từng phòng|sẵn|Phun|full|toàn bộ|gắn tường|liên tường|liền tường|đủ|cẩn thận|hoàn thiện nội thất', case=False, na=False)
data.loc[wrong_form, 'Nội thất'] = 3 #'Đầy đủ'

wrong_form = furniture.str.contains(r'Xây thô|Thô|Không nội thất', case=False, na=False)
data.loc[wrong_form, 'Nội thất'] = 1 #'Không nội thất' 


In [17]:
data['Nội thất'].drop_duplicates()

0     3
1     1
2     2
58    4
Name: Nội thất, dtype: object

In [18]:
project_name = data['Tên dự án']
wrong_form = project_name.str.contains('Không có thông tin', case=False, na=False)
data.loc[wrong_form, 'Tên dự án'] = "Không phải dự án"

investment = data['Chủ đầu tư']
wrong_form = investment.str.contains('Không có thông tin', case=False, na=False)
data.loc[wrong_form, 'Chủ đầu tư'] = 'Không có chủ đầu tư'


In [19]:
front = data['Mặt tiền']
wrong_form = front.str.contains(' m', case=False, na=False)
cleaned_wrong_form = data.loc[wrong_form, 'Mặt tiền'].str.replace(' m', '').str.replace(',', '.').astype(np.float32)
data.loc[wrong_form, 'Mặt tiền'] = cleaned_wrong_form

### Chuẩn bị dữ liệu ghi vào file đích

In [20]:
import re
def extract_info(column):
    extracted_data = []
    for item in column:
        if isinstance(item, str):
            # Trích xuất đường vào
            road_width = re.search(r'Đường vào:\s*([\d.,]+)\s*m', item)
            road_width_value = float(road_width.group(1).replace(',', '.')) if road_width else None

            # Trích xuất số tầng
            floors = re.search(r'Số tầng:\s*(\d+)', item)
            floors_value = int(floors.group(1)) if floors else None

             # Trích xuất số tòa
            buildings = re.search(r'Số tòa:\s*([\d.,]+)', item)
            buildings_value = int(buildings.group(1).replace('.', '')) if buildings else None

            # Trích xuất số căn hộ
            houses = re.search(r'Số căn hộ:\s*([\d.,]+)', item)
            houses_value = int(houses.group(1).replace('.', '')) if houses else None
            # Thêm vào danh sách
            extracted_data.append({'Đường vào': road_width_value, 'Số tầng': floors_value, 'Số tòa': buildings_value, 'Số căn hộ': houses_value})
        else:
            extracted_data.append({'Đường vào': None, 'Số tầng': None, 'Số tòa': None, 'Số căn hộ': None})
    return extracted_data

# Trích xuất thông tin từ "Thông tin khác"
info = extract_info(data['Thông tin khác'])

# Tạo DataFrame từ kết quả trích xuất
info_df = pd.DataFrame(info)


In [21]:
# Kết hợp tất cả dữ liệu
cleaned_data = pd.DataFrame({
    'Xã/Phường': data['Xã/Phường'],
    'Quận/Huyện': data['Quận/Huyện'],
    'Chủ đầu tư': data['Chủ đầu tư'],
    'Tên dự án': data['Tên dự án'],
    'Loại hình': data['Phân loại'],
    'Diện tích': area,
    'Mức giá': price,
    'Số phòng ngủ': bed_room,
    'Số toilet': toilet_room,
    'Pháp lý': legal,
    'Nội thất': furniture,
    'Mặt tiền': data['Mặt tiền'],
    'Đường vào': info_df['Đường vào'],
    'Số tầng': info_df['Số tầng'],
    'Số tòa': info_df['Số tòa'],
    'Số căn hộ': info_df['Số căn hộ'],
    'Hướng nhà': data['Hướng nhà'],
    'Hướng ban công': data['Hướng ban công'],
    'Lịch sử giá': data['Lịch sử giá'],
    'Khoảng giá': data['Khoảng giá']
})

In [22]:
cleaned_data['Xã/Phường'].str.contains(r'Xã/Phường/Thị Trấn', na=False).any()


np.False_

In [23]:
cleaned_data['Quận/Huyện'].str.contains(r'Quận|Huyện', na=False).any()

np.False_

### Mã hóa dữ liệu giá

In [24]:
month_price_path = '../../Data/priceData/month_price.csv'
quarter_price_path = '../../Data/priceData/quarter_price.csv'

history_price = pd.DataFrame(index=cleaned_data.index)
price_data = cleaned_data[['Lịch sử giá', 'Khoảng giá']].copy()

for idx, (history, price_range_str) in price_data.iterrows():
    entries = history.split('; ')
    price_ranges = price_range_str.split('; ')
    
    for i, entry in enumerate(entries):
        try:
            period, price = entry.split(': ')
            price_value = float(price.replace(' tr/m²', '').replace(',', '.'))
            
            min_price, max_price = price_ranges[i].split()
            min_value = float(min_price.replace(',', '.'))
            max_value = float(max_price.replace(',', '.'))

            history_price.loc[idx, period.strip()] = f"{min_value} {price_value} {max_value}"
            
        except:
            pass

month_price = history_price[history_price['Giá Q3/24'].isna() & history_price['Giá T10/24'].notna()].drop(
    columns=['Giá Q3/22', 'Giá Q4/22', 'Giá Q1/23', 'Giá Q2/23', 'Giá Q3/23', 'Giá Q4/23', 'Giá Q1/24', 'Giá Q2/24', 'Giá Q3/24']
)

quarter_price = history_price[history_price['Giá Q3/24'].notna()].drop(
    columns=['Giá T10/22', 'Giá T11/22', 'Giá T12/22', 'Giá T1/23', 'Giá T2/23', 'Giá T3/23', 'Giá T4/23', 'Giá T5/23', 'Giá T6/23', 'Giá T7/23', 'Giá T8/23', 'Giá T9/23', 'Giá T10/23', 'Giá T11/23', 'Giá T12/23', 'Giá T1/24', 'Giá T2/24', 'Giá T3/24', 'Giá T4/24', 'Giá T5/24', 'Giá T6/24', 'Giá T7/24', 'Giá T8/24', 'Giá T9/24', 'Giá T10/24']
)

none_price = history_price[history_price['Giá Q3/24'].isna() & history_price['Giá T10/24'].isna()]

month_indices = month_price.index
quarter_indices = quarter_price.index
none_indices = none_price.index

cleaned_data.loc[month_indices, 'Lịch sử giá'] = 'M'
cleaned_data.loc[quarter_indices, 'Lịch sử giá'] = 'Q'
cleaned_data.loc[none_indices, 'Lịch sử giá'] = 'N'

cleaned_data.drop(columns=['Khoảng giá'], inplace=True)
cleaned_data.rename(columns={"Lịch sử giá" : "Mã lịch sử giá"}, inplace=True)

### Ghi vào các file đích

In [25]:
cleaned_data.to_csv(path_data_goal, index=False)
month_price.to_csv(month_price_path, index=True, index_label="index")
quarter_price.to_csv(quarter_price_path, index=True, index_label="index")

### Mô tả dữ liệu

In [33]:
pd.set_option('display.max_rows', None)      # Hiển thị tất cả các hàng
pd.set_option('display.max_columns', None)   # Hiển thị tất cả các cột
pd.set_option('display.max_colwidth', None)  # Không giới hạn độ rộng của cột

In [ ]:
print(cleaned_data.info())

----- Tổng quan dữ liệu -----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11240 entries, 0 to 11239
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Xã/Phường       11240 non-null  object 
 1   Quận/Huyện      11240 non-null  object 
 2   Chủ đầu tư      11240 non-null  object 
 3   Tên dự án       11240 non-null  object 
 4   Loại hình       11240 non-null  object 
 5   Diện tích       11240 non-null  float32
 6   Mức giá         11240 non-null  object 
 7   Số phòng ngủ    11240 non-null  float32
 8   Số toilet       11240 non-null  float32
 9   Pháp lý         11240 non-null  object 
 10  Nội thất        11240 non-null  object 
 11  Mặt tiền        11240 non-null  object 
 12  Đường vào       5248 non-null   float64
 13  Số tầng         7586 non-null   float64
 14  Số tòa          1951 non-null   float64
 15  Số căn hộ       1267 non-null   float64
 16  Hướng nhà       11240 non-null  object 
 17  H

In [ ]:
print(cleaned_data.describe(include=[float, int]))


----- Mô tả dữ liệu số -----
            Diện tích  Số phòng ngủ     Số toilet    Đường vào      Số tầng  \
count    11240.000000  11240.000000  11240.000000  5248.000000  7586.000000   
mean       208.296844      4.751868      4.508452    11.540111     4.729238   
std       9449.311523      7.292260      7.351176    12.082593     1.775088   
min         10.000000      1.000000      1.000000     0.500000     1.000000   
25%         46.000000      3.000000      2.000000     4.000000     4.000000   
50%         70.000000      4.000000      3.000000     8.000000     5.000000   
75%        105.000000      5.000000      5.000000    15.000000     5.000000   
max    1000000.000000    255.000000    255.000000   265.000000    30.000000   

            Số tòa     Số căn hộ  
count  1951.000000   1267.000000  
mean     15.586879   2162.244672  
std      21.265465   2310.593738  
min       1.000000    104.000000  
25%       3.000000    668.000000  
50%       5.000000   1544.000000  
75%      17.0

In [ ]:
print(cleaned_data.describe(include=[object]))


----- Mô tả dữ liệu phân loại -----
         Xã/Phường Quận/Huyện           Chủ đầu tư         Tên dự án  \
count        11240      11240                11240             11240   
unique         342         30                  228               456   
top     Nhân Chính    Hà Đông  Không có chủ đầu tư  Không phải dự án   
freq           289       1448                 7126              7126   

        Loại hình  Mức giá  Pháp lý  Nội thất            Mặt tiền  \
count       11240  11240.0    11240     11240               11240   
unique         11   5738.0        4         4                 298   
top     Nhà riêng    200.0        4         3  Không có thông tin   
freq         4493     87.0    10522      5969                5215   

                 Hướng nhà      Hướng ban công Mã lịch sử giá  
count                11240               11240          11240  
unique                   9                   9              3  
top     Không có thông tin  Không có thông tin              M  
